### 목적
- LLM이 작업을 수행할 수 있도록 정보와 도구를 적절한 형식으로 제공하는 동적 시스템 구축

#### 컨텍스트 엔지니어링 vs 프롬프트 엔지니어링
| 구분    | 프롬프트 엔지니어링                 | 컨텍스트 엔지니어링                                        |
| ----- | -------------------------- | ------------------------------------------------- |
| 초점    | 한 번의 입력문 설계                | 전체 대화·상태·환경의 구조 설계                                |
| 단위    | 문자열 prompt                 | 구조화된 context object                               |
| 접근 방식 | “어떻게 물어볼까?”                | “무엇을 알고, 어떤 상황에서 대답해야 하나?”                        |
| 주요 기술 | few-shot, CoT, role prompt | context_schema, dynamic_prompt, middleware, state |
| 결과    | 즉시 품질 향상                   | 지속적 일관성, 개인화, 멀티턴 정확도                             |
| 비유    | 문장 기술                      | 시스템 설계                                            |


#### 컨텍스트 종류
| 컨텍스트 종류                | 목적                                      | 어디에 저장되나   | 지속 시간                |
| ---------------------- | --------------------------------------- | ---------- | -------------------- |
| **Model Context**      | 모델에게 주는 일시적 정보 (프롬프트, 메시지, 툴 목록 등)      | 요청 내부      | 🔹짧음 (한 번 호출용)       |
| **Tool Context**       | 툴 함수가 읽고 쓸 수 있는 환경(context/state/store) | Runtime 객체 | 🔸중간 (세션 내 유지)       |
| **Life-cycle Context** | 실행 전후 감시·제어용 (미들웨어, 요약, 검열 등)           | 미들웨어 계층    | 🔸지속 (여러 호출 간 유지 가능) |

- Model Context: 매 호출마다 새로 생성 (Transient)
- Tool/Life-cycle Context: 실행 중 혹은 장기적으로 유지 (Persistent)

#### 컨텍스트 데이터 구성
- context: 현재 사용자나 환경같은 고정 정보
    - ex) user_id, language, timezone
- state: 대화 중 잠깐 유지되는 단기 메모리
    - ex) 직전 메시지, 업로드 파일 요약
- store: 장기 메모리
    - ex) 사용자 선호, 과거 대화 요약, 영구 기록

#### 각 컨텍스트 별로 다루는 데이터 소스
| 컨트롤 계층                 | 읽는 데이터                      | 쓰는 데이터          | 주요 접근 방식                                                 | 비고          |
| ---------------------- | --------------------------- | --------------- | -------------------------------------------------------- | ----------- |
| **Model Context**      | `context`, `state`, `store` | ❌ (읽기 중심)       | `request.context`, `request.state`, `request.store`      | 모델 호출 직전/직후 |
| **Tool Context**       | `state`, `store`            | ✅ (Command로 갱신) | `runtime.state`, `runtime.store`                         | 툴 함수 내 실행   |
| **Life-cycle Context** | `state`, `store`            | ✅ (요약/기록)       | `before_model`, `after_model`, `SummarizationMiddleware` | 대화 관리/압축    |


#### Model Context 조작 기법 - System prompt
- @dynamic_prompt
    - 실행 시점에 state, runtime.context, runtime.store를 읽어 system prompt를 생성

In [56]:
# state 사용 예시 코드
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

@dynamic_prompt
def system_prompt_selector(request: ModelRequest) -> str:
    """
    현재 대화 길이에 따라 system prompt를 자동 변경한다.
    - 메시지가 많으면: 간결하게
    - 메시지가 적으면: 일반 모드
    """
    # request.state["messages"] 를 request.messages 로 줄여서 적을 수 있다. -> 최신 버전 권장은 축약 형태
    # message_count = len(request.messages)
    message_count = len(request.state["messages"])
    print(len(request.state["messages"]))
    base = "너는 유용한 어시스턴트이다."
    if message_count > 5:
        base += "\n대화가 길어졌으니 간결하게 답변해라."
    else:
        base += "\n상세하고 친절하게 설명해라."
    return base

agent = create_agent(
    model=llm,
    tools=[],  
    middleware=[system_prompt_selector]
)

conversation_short = {"messages": [{"role": "user", "content": "인공지능이 뭐야?"}]}
conversation_long = {"messages": [{"role": "user", "content": "인공지능이 뭐야?"} for i in range(7)]}

print("▶ 짧은 대화:")
result_short = agent.invoke(conversation_short)
print(result_short["messages"][-1].content, "\n")

print("▶ 긴 대화:")
result_long = agent.invoke(conversation_long)
print(result_long["messages"][-1].content)

▶ 짧은 대화:
1
인공지능(AI, Artificial Intelligence)은 기계나 컴퓨터 시스템이 인간의 지능을 모방하거나 이를 넘어서기 위해 설계된 기술과 알고리즘을 의미합니다. 인공지능은 다양한 분야에서 활용되며, 기본적으로 사람처럼 학습하고, 문제를 해결하고, 의사 결정을 내리는 능력을 가지도록 개발됩니다.

AI는 크게 두 가지 주요 범주로 나눌 수 있습니다:

1. **협의의 인공지능(Narrow AI)**: 특정 작업을 수행하는 데 특화된 AI입니다. 예를 들어, 이미지 인식, 자연어 처리, 음성 인식, 게임 플레이 등 특정한 문제를 해결하는 데 매우 뛰어난 성능을 냅니다. 이러한 AI는 우리가 일상에서 흔히 사용하는 챗봇이나 추천 시스템 등에 많이 사용됩니다.

2. **일반 인공지능(General AI)**: 인간과 같은 수준의 지능을 가지며, 여러 가지 종류의 작업을 이해하고 수행할 수 있는 AI를 의미합니다. 현재 일반 인공지능은 아직 개발되지 않았지만, 이는 미래의 목표 중 하나입니다.

인공지능의 주요 구성 요소로는 기계 학습(Machine Learning), 딥 러닝(Deep Learning), 자연어 처리(Natural Language Processing) 등이 있습니다.

- **기계 학습**: 데이터를 통해 학습하여 예측하거나 결정을 내리는 알고리즘입니다. 데이터가 많을수록 더 나은 예측을 할 수 있습니다.
- **딥 러닝**: 인공 신경망(Artificial Neural Networks)의 발전된 형태로, 대량의 데이터를 처리하고 복잡한 패턴을 인식하는 데 강력한 성능을 발휘합니다.
- **자연어 처리**: 컴퓨터가 인간의 언어를 이해하고 해석하는 기술로, 인간과의 의사소통을 가능하게 합니다.

인공지능은 의료, 금융, 제조, 자율주행 차량, 교육 등 다양한 분야에서 활용되며, 미래 사회에 큰 영향을 미칠 것으로 예상됩니다. 

▶ 긴 대화:
7
인공지능(AI)은 기계나 소프트웨어가 인간처럼 학습, 추론, 문제 해결을 할 

In [ ]:
# runtime.context와 runtime.store 사용 예시 코드

import os
from dataclasses import dataclass
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langgraph.store.memory import InMemoryStore

load_dotenv()

@dataclass
class Context:
    user_id: str

# 장기 스토어 준비
store = InMemoryStore()
store.put(("preferences",), "u-001", {"communication_style": "반말"})

# 동적 시스템 프롬프트: context.user_id로 store 조회
@dynamic_prompt
def store_aware_prompt(request: ModelRequest) -> str:
    user_id = request.runtime.context.user_id # 런타임에서 context로 들어온 user_id
    print("user id:", user_id)
    item = request.runtime.store.get(("preferences",), user_id) # 장기 메모리에서 user_id로 선호 검색
    base = "너는 유용한 조수이다."
    if item:
        style = item.value.get("communication_style", "균형잡힌 어투")
        base += f"\n유저의 선호도는 {style} 이다."
    return base

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[store_aware_prompt],
    context_schema=Context,
    store=store,
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "간단히 요약해서 인공지능 정의해줘"}]},
    context=Context(user_id="u-001"),
)

print(result["messages"][-1].content)

user id: u-001
인공지능은 인간의 지능을 모방하거나 대체할 수 있는 컴퓨터 시스템이나 프로그램이야. 주로 학습, 문제 해결, 패턴 인식 같은 기능을 통해 데이터에서 통찰을 얻고 의사 결정을 내리는 데 사용돼.


#### Model Context 조작 기법 - Messages
- @wrap_model_call
    - 모델에 실제로 들어가는 메시지 리스트를 조작 -> 모델 입력을 직전에 다듬어 정확도, 안전성, 토큰 효율을 높인다.(요약, 정리, 필터링, 컨텍스트 주입 등)

In [ ]:
import os
from typing import Callable, List, Dict, Any

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.agents import AgentState

load_dotenv()

class MyState(AgentState):
    uploaded_files: List[Dict[str, Any]]

@wrap_model_call
def inject_file_context(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """이 세션에서 사용자가 업로드한 파일에 대한 컨텍스트를 삽입합니다."""
    uploaded_files_state: List[Dict[str, Any]] = request.state.get('uploaded_files', [])
    
    if uploaded_files_state:
        file_description = [
            f"{f['name']} ({f['type']}): {f['summary']}"
            for f in uploaded_files_state
        ]

        file_context = (
            "이 대화에서 사용 가능한 파일:\n"
            + "\n".join(file_description)
            + "\n답변할 때 이것을 참고 자료로 활용하세요."
        )

        augmented_messages = [
            {'role': 'system', 'content': file_context},
            *request.messages # 스프레드(unpack) 문법 -> request.messages을 풀어서 뒤에 이어붙임
        ]

        request = request.override(messages=augmented_messages) # 수정한 messages를 교체한다.
        # override()를 써야하는 이유?
        # ModelRequest 객체는 불변(immutable)이라서 request.messages=... 로 수정할 수 없다.
    
    return handler(request)

uploaded_files_state = [
    {"name": "project_plan.pdf", "type": "pdf", "summary": "Q4 마일스톤/데드라인 요약"},
    {"name": "error_log.txt", "type": "text", "summary": "최근 에러와 타임스탬프"},
]

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[inject_file_context],
    state_schema=MyState
)

result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "최근 로그를 참고해서 서비스 장애 원인 요약해줘."}
        ],
        "uploaded_files": uploaded_files_state
    },
)

print(result["messages"][-1].content)

최근 로그를 확인해보겠습니다. 주어진 `error_log.txt` 파일을 참고하여 서비스 장애의 원인을 요약하겠습니다. 

파일을 열어봐야 할 내용이므로, 직접적으로 내용을 분석할 수는 없지만, 일반적으로 에러 로그에는 다음과 같은 정보가 포함되어 있을 수 있습니다:

1. **타임스탬프**: 에러가 발생한 시각.
2. **에러 코드**: 어떤 종류의 에러인지 알 수 있는 코드.
3. **메시지**: 에러에 대한 상세 설명.
4. **발생 위치**: 코드의 어떤 부분에서 문제가 발생했는지.

이 정보를 바탕으로 최근 에러가 발생한 시각, 에러 종류, 에러 메시지 등을 종합하여 장애의 원인을 분석할 수 있습니다.

직접적인 원인 분석이 필요하시면 해당 `error_log.txt` 파일의 내용을 직접 확인해주시면, 더 구체적으로 도움을 드릴 수 있습니다. 추가적으로 궁금한 사항이 있으면 말씀해 주세요.


In [53]:
import os
from dataclasses import dataclass
from typing import Callable

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langgraph.store.memory import InMemoryStore

load_dotenv()

@dataclass
class Context:
    user_id: str

@wrap_model_call
def inject_writing_style(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """Inject user's email writing style from Store."""
    user_id = request.runtime.context.user_id

    store = request.runtime.store
    writing_style = store.get(("writing_style",), user_id)

    if writing_style:
        style = writing_style.value
        style_context = f"""Your writing style:
        - Tone: {style.get('tone', 'professional')}
        - Typical greeting: "{style.get('greeting', 'Hi')}"
        - Typical sign-off: "{style.get('sign_off', 'Best')}"
        - Example email you've written:
        {style.get('example_email', '')}"""
        messages = [
            *request.messages,
            {"role": "user", "content": style_context}
        ]
        request = request.override(messages=messages)

    return handler(request)

llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# 장기 메모리 준비
store = InMemoryStore()
store.put(("writing_style",), "u-001", {
    "tone": "friendly and concise",
    "greeting": "안녕하세요",
    "sign_off": "감사합니다",
    "example_email": "안녕하세요, 어제 요청주신 로그 분석 결과를 공유드립니다. 핵심 원인은 ...",
})

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[inject_writing_style],
    context_schema=Context,
    store=store,
)

result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "고객 A에게 장애 원인과 재발 방지 조치에 대한 이메일 초안 작성해줘."}
        ]
    },
    context=Context(user_id="u-001")
)

print(result["messages"][-1].content)


안녕하세요, 고객 A님.

최근 발생한 장애에 대한 원인과 재발 방지 조치에 대해 안내드립니다.

장애의 주요 원인은 시스템 내에서 예기치 않은 오류로 인한 것이었습니다. 이를 통해 고객님의 서비스에 불편을 드린 점 진심으로 사과드립니다. 

재발 방지를 위해 다음과 같은 조치를 취할 예정입니다:
1. 시스템 모니터링 강화: 오류 발생 시 즉시 알림을 받을 수 있도록 모니터링 시스템을 개선하겠습니다.
2. 버그 수정: 이번 이슈와 관련된 코드를 점검하고 수정하여 유사 사례가 발생하지 않도록 하겠습니다.
3. 정기적인 점검: 향후 정기적인 시스템 점검을 실시하여 문제를 사전에 발견할 수 있도록 하겠습니다.

혹시 추가로 궁금하신 사항이 있으시면 언제든지 말씀해 주세요.

감사합니다.


#### Model Context 조작 기법 - Tools
- 툴에는 명확한 이름, 설명, 인수 이름, 인수 설명이 필요하다. 
    - 단순히 메타데이터가 아니라 LLM이 도구를 사용하기 위한 모델의 추론을 안내하는 역할이다.
- 툴이 너무 많으면 모델에 과부하(컨텍스트 과부하)를 일으켜 오류가 증가하고, 도구가 너무 적으면 기능이 제한된다.
    - 사용 가능한 도구 세트를 조정하기 위해 Model Context(state, store, context)를 이용한다.
    - 예) 인증 전에는 public tool만 노출, 인증 후에는 private tool 활성화 등